In [ ]:
import numpy as np
import pandas as pd
import random
import pulp

In [ ]:
alldish = pd.read_csv('finaldata.csv')

In [ ]:
nutrition = alldish[['title', 'calories', 'sodium', 'fat', 'protein']]

In [ ]:
meal = list(range(5))
person = ['1', '2', '3', '4']
dish = nutrition2['title'].tolist()

In [ ]:
rating = pd.read_csv('rating.csv')
title = nutrition['title'].values.tolist()
rating = rating['title']
price = alldish[['title', 'price']]
cooking_time = alldish[['title', 'time']]
availability_matrix = [[90, 240, 180, 90, 0],
                       [60, 20, 130, 0, 240],
                       [240, 240, 0, 75, 180],
                       [60, 0, 60, 360, 240]]

In [ ]:
def set_params_basic(meal, person, dish, 
                     rating, nutrition, price, availability_matrix, cooking_time):    
    r = rating.values.tolist()
    r = pulp.makeDict([person, dish], r, 0)
    p = price['price'].values.tolist()
    p = pulp.makeDict([dish], p, 0)
    budget = 300
    price_budget = [p, budget]
    z = availability_matrix
    z = pulp.makeDict([person, meal], z, 0)
    n_sodium = nutrition['sodium'].tolist()
    n_sodium = pulp.makeDict([dish], n_sodium, 0)
    up_sodium = 2300 * 4 
    low_sodium = 0    
    n_calory = nutrition['calories'].tolist()
    n_calory = pulp.makeDict([dish], n_calory, 0)
    up_calory = 2000 * 4 
    low_calory = 0
    n_fat = nutrition['fat'].tolist()
    n_fat = pulp.makeDict([dish], n_fat, 0)
    up_fat = 35 * 4 
    low_fat = 25 * 4     
    n_protein = nutrition['protein'].tolist()
    n_protein = pulp.makeDict([dish], n_protein, 0)
    up_protein = 46 * 4
    low_protein = 0
    nutri = {'sodium': [n_sodium, up_sodium, low_sodium],
             'calory': [n_calory, up_calory, low_calory],
             'fat': [n_fat, up_fat, low_fat],
             'protein': [n_protein, up_protein, low_protein]}
    T = cooking_time['time'].values.tolist()
    T = pulp.makeDict([dish], T, 0)    
    ls_params_basic = [r, nutri, price_budget, a, z, T]
    
    return ls_params_basic

In [ ]:
def creative_model(meal, person, dish, ls_params_basic):
    nm = len(meal)
    ns = len(person)
    nd = len(dish)
    r = ls_params_basic[0]
    nutri = ls_params_basic[1]
    price_budget = ls_params_basic[2]
    a = ls_params_basic[3]
    z = ls_params_basic[4]
    T = ls_params_basic[5]    
    meals = pulp.LpProblem("Meal_Plan", pulp.LpMaximize)
    d = pulp.LpVariable.dicts("dish", (dish, meal), 0, 1, pulp.LpInteger)
    y = pulp.LpVariable.dicts("cook", (p, meal), 0, 1, pulp.LpInteger)
    meals += (pulp.lpSum([r[i][j] * d[j][k] for i in p for k in meal for j in dish] + C), 'sum_of_rating')
    
    for i in perosn:
        for j in dish:
            meals += (rt.iloc[i,j] <= C)

    for k in meal:
        meals += (pulp.lpSum([d[j][k] for j in dish]) <= 1)

    for j in dish:
        meals += (pulp.lpSum([d[j][k] for k in meal]) <= 1)

    for k in range(5):
        meals += (
            pulp.lpSum([nutri['sodium'][0][j] * d[j][k] for j in dish]) <= nutri['sodium'][1]
        )
        meals += (
            pulp.lpSum([nutri['sodium'][0][j] * d[j][k] for j in dish]) >= nutri['sodium'][2]
        )

        meals += (
            pulp.lpSum([nutri['calory'][0][j] * d[j][k] for j in dish]) <= nutri['calory'][1]
        )
        meals += (
            pulp.lpSum([nutri['calory'][0][j] * d[j][k] for j in dish]) >= nutri['calory'][2]
        )

        meals += (pulp.lpSum([nutri['fat'][0][j] * d[j][k] for j in dish]) <= nutri['fat'][1])
        meals += (pulp.lpSum([nutri['fat'][0][j] * d[j][k] for j in dish]) >= nutri['fat'][2])

        meals += (
            pulp.lpSum([nutri['protein'][0][j] * d[j][k] for j in dish]) <= nutri['protein'][1]
        )
        meals += (
            pulp.lpSum([nutri['protein'][0][j] * d[j][k] for j in dish]) >= nutri['protein'][2]
        )
        

    meals += (pulp.lpSum([price_budget[0][j] * d[j][k] for j in dish for k in meal]) <= price_budget[1])

    for i in person:
        for k in meal:
            meals += (y[i][k] <= z[i][k])

    for k in meal:
        meals += (pulp.lpSum([y[i][k] for i in person]) <= 1)

    for i in p:
        meals += (pulp.lpSum([y[i][k] for k in meal]) <= 2)
        meals += (pulp.lpSum([y[i][k] for k in meal]) >= 1)

    for k in meal:
        for j in dish:
            meals += (T[j] * d[j][k] <= pulp.lpSum([y[i][k] * z[i][k] for i in p]))
            
    status = meals.solve()
    
    return meals

In [ ]:
ls_params_basic = set_params_basic(meal, person, dish, 
                                   rating, nutrition, price, availability_matrix, cooking_time)
meals_basic = basic_model(meal, p, dish, ls_params_basic)